# RFC 001: Global run function

In [ ]:
from abc import ABC
from random import randint
from typing import Annotated, Any, Iterable, Literal, Optional, Protocol, Union, runtime_checkable
from unittest.mock import MagicMock
from uuid import UUID, uuid4

from pydantic import BaseModel, Field

from autogen import Agent as OriginalAgent
from autogen import ConversableAgent as OriginalConversableAgent
from autogen.tools import Depends, Tool, tool


def on(o: Any):
    return lambda o=o: o

## Prerequisites


### LLMConfig

In [ ]:
@runtime_checkable
class LLMConfigProtocol(Protocol):
    def __enter__(self): ...

    def __exit__(self, exc_type, exc_value, traceback): ...


class LLMConfig:
    llm_config: list[dict[str, Any]] = []

    def __init__(self, llm_config: dict[str, Any]):
        self._llm_config = llm_config

    def __enter__(self):
        LLMConfig.llm_config.append(self._llm_config)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        LLMConfig.llm_config.pop()

    @classmethod
    def get_llm_config(cls):
        return cls.llm_config[-1] if LLMConfig.llm_config else False

### Conversable agent

In [ ]:
class ConversableAgent(OriginalConversableAgent):
    def __init__(self, *args, **kwargs):
        kwargs["name"] = kwargs.get("name", f"name-{randint(0, 1000)}")
        llm_config = kwargs.get("llm_config", {}) if "llm_config" in kwargs else LLMConfig.get_llm_config()
        kwargs["llm_config"] = llm_config
        super().__init__(*args, **kwargs)

    def add_tool(self, tool: Tool):
        tool.register_for_llm(self)


ConversableAgent()

### Agent

In [ ]:
class AgentProtocol(OriginalAgent, Protocol):
    @property
    def run_info(self) -> Optional["RunInfo"]: ...

In [ ]:
def create_sub_run_tool(agent: AgentProtocol) -> Tool:
    @tool()
    def sub_chat(agent: Annotated[AgentProtocol, Depends(on(agent))]) -> str:
        agent1 = MagicMock(spec=ConversableAgent)
        agent2 = MagicMock(spec=ConversableAgent)

        response = run(agent1, agent2, above_run=agent.run_info, message="Hello, world!")

        return response.summary

    return sub_chat


# agent = MagicMock(spec=AgentProtocol)
# create_sub_chat(agent).func()

### Messages

Eventually, we will move away from dictionaries and represent all messages with objects (dataclasses or Pydantic base models). For now, we will use existing implementation.

In [ ]:
Message = dict[str, Any]

### Events

These are six different abstract types of events used by the runtime.

In [ ]:
EventType = Literal["input_request", "async_input_request", "input_response", "agent_message", "output", "system"]


class Event(BaseModel, ABC):
    uuid: Annotated[UUID, Field(default_factory=uuid4)]

    type: EventType


class InputRequestEvent(Event):
    prompt: str

    def respond(self, response: "InputResponseEvent"):
        pass

    type: EventType = "input_request"


class AsyncInputRequestEvent(Event):
    prompt: str

    async def a_respond(self, response: "InputResponseEvent"):
        pass

    type: EventType = "async_input_request"


class InputResponseEvent(Event):
    type: EventType = "input_response"

    value: str


class AgentMessageEvent(Event):
    message: Message

    type: EventType = "agent_message"


class OutputEvent(Event):
    value: str

    type: EventType = "output"


class SystemEvent(Event):
    value: str

    type: EventType = "system"


class ErrorEvent(Event):
    type: EventType = "error"

    error: str

## Global run function

### Run response object

In [ ]:
from typing import AsyncIterable


class RunInfoProtocol(Protocol):
    @property
    def uuid(self) -> UUID: ...

    @property
    def above_run(self) -> Optional["RunResponseProtocol"]: ...


class RunResponseProtocol(RunInfoProtocol, Protocol):
    @property
    def events(self) -> Iterable[Event]: ...

    @property
    def messages(self) -> Iterable[Message]: ...

    @property
    def summary(self) -> str: ...


class AsyncRunResponseProtocol(RunInfoProtocol, Protocol):
    @property
    def uuid(self) -> UUID: ...

    @property
    def above_run(self) -> Optional["RunResponseProtocol"]: ...

    @property
    def events(self) -> AsyncIterable[Event]: ...

    @property
    def messages(self) -> AsyncIterable[Message]: ...

    @property
    async def summary(self) -> str: ...

### Chat manager protocol

In [ ]:
class ChatManagerProtocol(Protocol):
    def register_agent(self, agent: Union[AgentProtocol, list[AgentProtocol]]) -> None: ...
    def get_next_agent(self, response: RunResponseProtocol) -> AgentProtocol: ...


class AsyncChatManagerProtocol(Protocol):
    def register_agent(self, agent: Union[AgentProtocol, list[AgentProtocol]]) -> None: ...
    async def a_get_next_agent(self, response: AsyncRunResponseProtocol) -> AgentProtocol: ...

### Round robin chat manager example

In [ ]:
class RoundRobinChatManager:
    def __init__(self):
        self.agents: list[AgentProtocol] = []
        self.agent_names: list[str] = []

    def register_agent(self, agent: Union[AgentProtocol, list[AgentProtocol]]) -> None:
        if isinstance(agent, list):
            self.agents.extend(agent)
            self.agent_names.extend([agent.name for agent in agent])
        else:
            self.agents.append(agent)
            self.agent_names.append(agent.name)

    def get_next_agent(self, response: RunResponseProtocol) -> AgentProtocol:
        # Get last agent
        if not response.messages:
            return self.agents[0]

        last_agent = response.messages[-1].message["sender"]
        last_agent_index = self.agent_names.index(last_agent)
        next_agent_index = (last_agent_index + 1) % len(self.agent_names)
        return self.agents[next_agent_index]

In [ ]:
from dataclasses import dataclass
from typing import Callable


@dataclass
class SummaryMethod:
    def __init__(
        self,
        method: Optional[
            Union[
                Literal["last_msg", "reflection_with_llm"],
                Callable[[AgentProtocol, AgentProtocol, dict[str, Any]], str],
            ]
        ] = None,
        **kwargs: Any,
    ):
        self.method = method if method else ConversableAgent.DEFAULT_SUMMARY_METHOD
        self.kwargs = kwargs

In [ ]:
def run(
    *agents: AgentProtocol,
    message: Optional[str] = None,
    continuation_of: Optional[RunResponseProtocol] = None,
    sub_run_of: Optional[RunInfoProtocol] = None,
    chat_manager: Optional[ChatManagerProtocol] = None,
    max_turns: Optional[int] = None,
    summary_method: Optional[SummaryMethod] = None,
    **kwargs: Any,
) -> RunResponseProtocol:
    """Run the agents with the given initial message.

    Args:
        agents: The agents to run.
        message: The initial message to send to the first agent.
        previous_run: The previous run to continue.
        kwargs: Additional arguments to pass to the agents.

    """
    ...


async def a_run(
    *agents: AgentProtocol,
    message: Optional[str] = None,
    continuation_of: Optional[AsyncRunResponseProtocol] = None,
    sub_run_of: Optional[RunInfoProtocol] = None,
    chat_manager: Optional[AsyncChatManagerProtocol] = None,
    max_turns: Optional[int] = None,
    summary_method: Optional[SummaryMethod] = None,
    **kwargs: Any,
) -> AsyncRunResponseProtocol:
    """Run the agents with the given initial message.

    Args:
        agents: The agents to run.
        message: The initial message to send to the first agent.
        previous_run: The previous run to continue.
        kwargs: Additional arguments to pass to the agents.

    """
    ...

## Event processing loop

In [ ]:
llm_config.get_llm_config()
with llm_config:
    print(LLMConfig.llm_config)
    print(llm_config.get_llm_config())

In [ ]:
import unittest
from os import environ

# llm_config will be an object that can be used as a context manager
# this will allow us to set the configuration for the agents without passing it as an argument
llm_config = LLMConfig({"config_list": [{"api_key": environ["OPENAI_API_KEY"], "model": "gpt-4o-mini"}]})

with llm_config:
    alice = ConversableAgent(system_message="You are a useful assistant doing your best to solve a task problem.")
    bob = ConversableAgent(
        system_message="You are a fact checker and your job is to check all fact using realible sources."
    )


# tools will be attached to agents directly without using register_for_llm and register_for_execution functions
@tool(description="Search Wikipedia for the given query.")
def search_wikipedia(query: str) -> str: ...


class Answer(BaseModel):
    answer: Annotated[str, "A short answer to the user's question."]
    explanation: Annotated[str, "A detailed explanation of the answer."]
    links_visited: list[str]


@tool(description="Send the answer to the user.")
def send_answer(answer: Answer) -> str: ...


alice.add_tool(send_answer)
bob.add_tool(search_wikipedia)

# finally, we will execute a global run function to run the agents
# run function will internally create a UserProxyAgent and use it for tool execution
# the function will return immediately and return an RunResponseProtocol object that can be used for iterating over events and messages produced by the agents running
# there are both sync and async versions of the run function
response = run(alice, bob, message="Who is Leonardo da Vinci?")


# mocking response
response = MagicMock(spec=RunResponseProtocol)
response.events = [
    InputRequestEvent(prompt="What is the meaning of life?"),
    InputResponseEvent(value="42"),
    AgentMessageEvent(message={"text": "What is the meaning of life?", "sender": "Alice"}),
    OutputEvent(value="thinking about it..."),
    SystemEvent(value="done"),
]
response.messages = [
    {"text": "What is the meaning of life?", "sender": "Alice"},
]
response.summary = "Alice and Bob had a conversation about the meaning of life."

with unittest.mock.patch("builtins.input", return_value="42"):
    for event in response.events:
        print(f"Event: {event}")
        if event.type == "input_request":
            print(f"Input request: {event.prompt}")
            s = input(event.prompt)

            print(f"Response from UI: {s}")
            event.respond(s)

        elif event.type == "input_response":
            # Do nothing, just pass
            if event.value != s:
                raise RuntimeError(f"Expected {s}, got {event.value}")

        elif event.type == "output":
            print(f"Output message: {event.value}")

        elif event.type == "agent_message":
            print(f"Agent message: {event.message}")

        elif event.type == "system":
            print(f"System event: {event}")

        elif event.type == "error":
            print(f"Error event: {event}")

print()
print(f"Messages: {response.messages}")

print()
print(f"Summary: {response.summary}")

## Examples runs

In [ ]:
def process_response(response: RunResponseProtocol):
    for event in response.events:
        print(f"Event: {event}")
        if event.type == "input_request":
            print(f"Input request: {event.prompt}")
            s = input(event.prompt)

            print(f"Response from UI: {s}")
            event.respond(s)

        elif event.type == "input_response":
            # Do nothing, just pass
            if event.value != s:
                raise RuntimeError(f"Expected {s}, got {event.value}")

        elif event.type == "output":
            print(f"Output message: {event.value}")

        elif event.type == "agent_message":
            print(f"Agent message: {event.message}")

        elif event.type == "system":
            print(f"System event: {event}")

        elif event.type == "error":
            print(f"Error event: {event}")

### Single agent

#### Single agent run simplest form (no functions, no conversation with the user)

In [ ]:
llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

agent = ConversableAgent(name="Alice", llm_config=llm_config)

response = run(agent, message="What is the meaning of life?", is_termination_message=lambda x: x == "42")

# mocking response
response = MagicMock(spec=RunResponseProtocol)
response.events = [
    AgentMessageEvent(message={"text": "thinking about it...", "sender": "Alice"}),
    AgentMessageEvent(message={"text": "42", "sender": "Alice"}),
    SystemEvent(value="done"),
]
response.messages = [
    {"text": "thinking about it...", "sender": "Alice"},
    {"text": "42", "sender": "Alice"},
]
response.summary = "Alice said the meaning of life is 42"

process_response(response)

print()
print(f"Messages: {response.messages}")

print()
print(f"Summary: {response.summary}")
process_response(response)

#### Single agent run using functions

In [ ]:
from autogen.tools import tool

llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}
agent = ConversableAgent(name="Alice", llm_config=llm_config)


@tool(description="Returns the meaning of life.")
def meaning_of_life():
    return "42"


meaning_of_life.register_tool(agent)
# agent.register_tool(meaning_of_life) # Alternative implementation

response = run(agent, message="What is the meaning of life?", is_termination_message=lambda x: x == "42")

# mocking response
response = MagicMock(spec=RunResponseProtocol)
response.events = [
    AgentMessageEvent(message={"text": "thinking about it...", "sender": "Alice"}),
    SystemEvent(value="Alice called meaning_of_life"),  # Replace WithFunctionCallEvent subclass of AgentMessageEvent
    SystemEvent(value="meaning_of_life returned 42"),
    AgentMessageEvent(message={"text": "42", "sender": "Alice"}),
    SystemEvent(value="done"),
]
response.messages = [
    {"text": "thinking about it...", "sender": "Alice"},
    {"text": "42", "sender": "Alice"},
]
response.summary = "Alice said the meaning of life is 42"

process_response(response)

print()
print(f"Messages: {response.messages}")

print()
print(f"Summary: {response.summary}")
process_response(response)

### Two agents

In [ ]:
# Chat between two comedian agents

# 1. Import our agent class
from autogen import ConversableAgent

# 2. Define our LLM configuration for OpenAI's GPT-4o mini,
#    uses the OPENAI_API_KEY environment variable
llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# 3. Create our agents who will tell each other jokes,
#    with Jack ending the chat when Emma says FINISH
jack = ConversableAgent(
    "Jack",
    llm_config=llm_config,
    system_message=("Your name is Jack and you are a comedian in a two-person comedy show."),
    is_termination_msg=lambda x: True if "FINISH" in x["text"] else False,
)
emma = ConversableAgent(
    "Emma",
    llm_config=llm_config,
    system_message=(
        "Your name is Emma and you are a comedian "
        "in a two-person comedy show. Say the word FINISH "
        "ONLY AFTER you've heard 2 of Jack's jokes."
    ),
)

response = run(jack, emma, message="Tell me a joke.")

# mocking response
response = MagicMock(spec=RunResponseProtocol)
response.events = [
    AgentMessageEvent(message={"text": "What do you call a fake noodle? An impasta.", "sender": "Jack"}),
    AgentMessageEvent(
        message={"text": "Haha, nice one! What do you call a belt made of watches? A waist of time.", "sender": "Emma"}
    ),
    AgentMessageEvent(
        message={"text": "Why couldn't the bicycle stand up by itself? It was two tired.", "sender": "Jack"}
    ),
    AgentMessageEvent(message={"text": "FINISH", "sender": "Emma"}),
    SystemEvent(value="done"),
]

response.messages = [
    {"text": "What do you call a fake noodle? An impasta.", "sender": "Jack"},
    {"text": "Haha, nice one! What do you call a belt made of watches? A waist of time.", "sender": "Emma"},
    {"text": "Why couldn't the bicycle stand up by itself? It was two tired.", "sender": "Jack"},
    {"text": "FINISH", "sender": "Emma"},
]

response.summary = "Jack and Emma had a comedy chat."

process_response(response)

### Group chat

In [ ]:
from autogen import ConversableAgent, GroupChat, GroupChatManager

llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# Planner agent setup
planner_message = "Create lesson plans for 4th grade."
planner = ConversableAgent(
    name="planner_agent", llm_config=llm_config, system_message=planner_message, description="Creates lesson plans"
)

# Reviewer agent setup
reviewer_message = "Review lesson plans against 4th grade curriculum. Provide max 3 changes."
reviewer = ConversableAgent(
    name="reviewer_agent", llm_config=llm_config, system_message=reviewer_message, description="Reviews lesson plans"
)

# Teacher agent setup
teacher_message = "Choose topics and work with planner and reviewer. Say DONE! when finished."
teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

groupchat = GroupChat(agents=[teacher, planner, reviewer], speaker_selection_method="auto", messages=[])

# Create manager
# At each turn, the manager will check if the message contains DONE! and end the chat if so
# Otherwise, it will select the next appropriate agent using its LLM
manager = GroupChatManager(
    name="group_manager",
    groupchat=groupchat,
    llm_config=llm_config,
    is_termination_msg=lambda x: "DONE!" in (x.get("content", "") or "").upper(),
)


class NewGroupChatManager(GroupChatManager):
    def __init__(self, is_termination_msg, llm_config):
        self._is_termination_msg = is_termination_msg
        self._llm_config = llm_config


response = run(planner, reviewer, teacher, message="Create lesson plans for 4th grade.", chat_manager=manager)

# mocking response
response = MagicMock(spec=RunResponseProtocol)

response.events = [
    AgentMessageEvent(message={"text": "Create lesson plans for 4th grade.", "sender": "planner_agent"}),
    AgentMessageEvent(
        message={
            "text": "Plan is: Math, Learn addition and subtraction, Script: Teach addition and subtraction using examples.",
            "sender": "planner_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "I would change the addition and subtraction with multiplication and division.",
            "sender": "reviewer_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "Plan is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
            "sender": "planner_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "Okay first lesson is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
            "sender": "teacher_agent",
        }
    ),
    AgentMessageEvent(message={"text": "DONE!", "sender": "teacher_agent"}),
    SystemEvent(value="done"),
]

response.messages = [
    {"text": "Create lesson plans for 4th grade.", "sender": "planner_agent"},
    {
        "text": "Plan is: Math, Learn addition and subtraction, Script: Teach addition and subtraction using examples.",
        "sender": "planner_agent",
    },
    {
        "text": "I would change the addition and subtraction with multiplication and division.",
        "sender": "reviewer_agent",
    },
    {
        "text": "Plan is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
        "sender": "planner_agent",
    },
    {
        "text": "Okay first lesson is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
        "sender": "teacher_agent",
    },
    {"text": "DONE!", "sender": "teacher_agent"},
]

response.summary = "planner_agent, reviewer_agent, and teacher_agent had a chat."

process_response(response)

In [ ]:
from autogen import ConversableAgent, GroupChat, GroupChatManager


class NewGroupChatManager(GroupChatManager):
    def __init__(self, is_termination_msg, llm_config):
        self._is_termination_msg = is_termination_msg
        self._llm_config = llm_config


llm_config = LLMConfig({"api_type": "openai", "model": "gpt-4o-mini"})


@tool
def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


with llm_config:
    planner = ConversableAgent("You are a planner. Collaborate with teacher and reviewer to create lesson plans.")

    reviewer = ConversableAgent(
        "You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes."
    )

    teacher = ConversableAgent(
        "You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    )
    teacher.add_tool(submit_plan)

    chat_manager = NewGroupChatManager(terminate_on=Keyword("DONE!"))

response = run(
    planner,
    reviewer,
    teacher,
    message="Create lesson plans for 4th grade.",
    chat_manager=chat_manager,
)

# mocking response
response = MagicMock(spec=RunResponseProtocol)

response.events = [
    AgentMessageEvent(message={"text": "Create lesson plans for 4th grade.", "sender": "planner_agent"}),
    AgentMessageEvent(
        message={
            "text": "Plan is: Math, Learn addition and subtraction, Script: Teach addition and subtraction using examples.",
            "sender": "planner_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "I would change the addition and subtraction with multiplication and division.",
            "sender": "reviewer_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "Plan is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
            "sender": "planner_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "Okay first lesson is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
            "sender": "teacher_agent",
        }
    ),
    AgentMessageEvent(message={"text": "DONE!", "sender": "teacher_agent"}),
    SystemEvent(value="done"),
]

response.messages = [
    {"text": "Create lesson plans for 4th grade.", "sender": "planner_agent"},
    {
        "text": "Plan is: Math, Learn addition and subtraction, Script: Teach addition and subtraction using examples.",
        "sender": "planner_agent",
    },
    {
        "text": "I would change the addition and subtraction with multiplication and division.",
        "sender": "reviewer_agent",
    },
    {
        "text": "Plan is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
        "sender": "planner_agent",
    },
    {
        "text": "Okay first lesson is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
        "sender": "teacher_agent",
    },
    {"text": "DONE!", "sender": "teacher_agent"},
]

response.summary = "planner_agent, reviewer_agent, and teacher_agent had a chat."

process_response(response)

In [ ]:
from autogen import ConversableAgent

llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# Planner agent setup
planner_message = "Create lesson plans for 4th grade."
planner = ConversableAgent(
    name="planner_agent", llm_config=llm_config, system_message=planner_message, description="Creates lesson plans"
)

# Reviewer agent setup
reviewer_message = "Review lesson plans against 4th grade curriculum. Provide max 3 changes."
reviewer = ConversableAgent(
    name="reviewer_agent", llm_config=llm_config, system_message=reviewer_message, description="Reviews lesson plans"
)

# Teacher agent setup
teacher_message = "Choose topics and work with planner and reviewer. Say DONE! when finished."
teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

response = run(
    planner,
    reviewer,
    teacher,
    message="Create lesson plans for 4th grade.",
    chat_manager=RoundRobinChatManager([planner, reviewer, teacher]),
)

# mocking response
response = MagicMock(spec=RunResponseProtocol)

response.events = [
    AgentMessageEvent(message={"text": "Create lesson plans for 4th grade.", "sender": "planner_agent"}),
    AgentMessageEvent(
        message={
            "text": "Plan is: Math, Learn addition and subtraction, Script: Teach addition and subtraction using examples.",
            "sender": "planner_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "I would change the addition and subtraction with multiplication and division.",
            "sender": "reviewer_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "Plan is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
            "sender": "planner_agent",
        }
    ),
    AgentMessageEvent(
        message={
            "text": "Okay first lesson is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
            "sender": "teacher_agent",
        }
    ),
    AgentMessageEvent(message={"text": "DONE!", "sender": "teacher_agent"}),
    SystemEvent(value="done"),
]

response.messages = [
    {"text": "Create lesson plans for 4th grade.", "sender": "planner_agent"},
    {
        "text": "Plan is: Math, Learn addition and subtraction, Script: Teach addition and subtraction using examples.",
        "sender": "planner_agent",
    },
    {
        "text": "I would change the addition and subtraction with multiplication and division.",
        "sender": "reviewer_agent",
    },
    {
        "text": "Plan is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
        "sender": "planner_agent",
    },
    {
        "text": "Okay first lesson is: Math, Learn multiplication and division, Script: Teach multiplication and division using examples.",
        "sender": "teacher_agent",
    },
    {"text": "DONE!", "sender": "teacher_agent"},
]

response.summary = "planner_agent, reviewer_agent, and teacher_agent had a chat."

process_run_response(response)

### Swarm

In [ ]:
sales_agent = ConversableAgent(name="Sales Agent", llm_config=llm_config)
complaints_agent = ConversableAgent(name="Complaints Agent", llm_config=llm_config)

triage_agent = ConversableAgent(name="Triage Agent", llm_config=llm_config)


@triage_agent.register_for_llm()
def transfer_to_sales():
    return sales_agent


@triage_agent.register_for_llm()
def transfer_to_complaints():
    return complaints_agent


response = run(triage_agent, message="I have a complaint about my order.")

# mocking response
response = MagicMock(spec=RunResponseProtocol)
response.events = [
    SystemEvent(value="Triage Agent called transfer_to_complaints"),
    SystemEvent(value="transfer_to_complaints returned Complaints Agent"),
    InputRequestEvent(prompt="Hi what is your complaint?"),
    InputResponseEvent(value="My order was late."),
    AgentMessageEvent(
        message={"text": "I'm sorry to hear that. We will make the order faster.", "sender": "Complaints Agent"}
    ),
    SystemEvent(value="done"),
]

response.messages = [
    {"text": "I'm sorry to hear that. We will make the order faster.", "sender": "Complaints Agent"},
]
response.summary = "Complaints Agent apologized for the late order."

with unittest.mock.patch("builtins.input", return_value="My order was late."):
    process_run_response(response, SimpleEventProcessor())